# **Documentation technique de l'application Dash**

## Nettoyage des données pour l'affichage

Dans un premier temps, nous avons nettoyé et recodé certaines variables afin de réaliser un affichage plus clair et plus agréable. Notamment, le sexe ('gender') était codé en "0", "1", que nous avons renommé "Hommes" et "Femmes". Nous avons de même réalisé un recodage sur les carrières "career_c".
Enfin, nous avons réalisé des classes, regroupements pour les différentes tranches d'âges.

Ces modifications se trouvent sur le fichier modele.py, qui recence les manipulation réalisées avant le traitement notre l'applciation Dash.

## Création d'une Sidebar

Nous avons créé une Sidebar, afin de naviguer entre nos différentes pages.
```
sidebar = html.Div(
    [
        html.Div(html.Img(src=logo,width='200',height='200',alt='logo'),style={'text-align':'center'}),
        html.Hr(),
        
        dbc.Nav([
            dbc.NavLink("Présentation des Profils", href="/PresentationdesProfils", active="exact"),html.Hr(),
            dbc.NavLink("Profil type", href="/ProfilType", active="exact"),html.Hr(),
            dbc.NavLink("Explication Modèle", href="/ExplicationModele", active="exact"),html.Hr(),
            ],vertical=True,pills=True,
        ),
   ],
   style=SIDEBAR_STYLE,
)
```
Ainsi, nous avons une partie "Présentation des Profils", une partie "Profil type" et une partie "Explication du modèle", où se trouve notamment les différentes variables explicatives de notre modèle de prédiction.

## Application Dash

Création de l'application Dash :

```
# Partie Contenu
content = html.Div(id="page-content", children=[], style=CONTENT_STYLE)

###############################################
# DASH Application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content
])
```

server = app.server permet de déployer l'application par la suite sur Heroku.
En dessous, le layout nous permet notre contenu, dont la Sidebar précédemment créée.

## Les fonctions callback

Ces fonctions permettent de réaliser des échanges de données. Il s'agit d'une fonction passée dans une autre fonction en tant qu'argument.

```
@app.callback(
    Output(component_id='boxplot_output', component_property='figure'),
    Input(component_id='radio_input', component_property='value'),
    Input(component_id='dropdown_input', component_property='value'))
def make_box(abs,ord):
    x_abs=data.loc[:,abs]
    y_ord=data.loc[:,ord]
    fig = go.Figure(data=[go.Box(x=x_abs,y=y_ord)])
    if (abs=='gender'):
        fig.update_layout(title_text="Importance de critères selon le sexe", title_x=0.5)
    elif (abs=='age'):
        fig.update_layout(title_text="Importance de critères selon l'âge", title_x=0.5)
    return fig
```
Cette première fonction callback permet de créer un boxplot, en prenant en entrée la valeur d'un radio bouton et d'une liste déroulante, et renvoie un graphique en sortie. Les entrées permettent de choisir les variables que nous voulons afficher. Les callbacks permettent donc des réaliser des graphiques interactifs.


```
@app.callback(
    Output(component_id='graph_output', component_property='figure'),
    Input(component_id='select_input', component_property='value'))
def get_data_table(option):
    val = decision.loc[:, option]
    fig = go.Figure(data=[go.Histogram(histfunc="avg",x=val,y=decision.iloc[:,0])])
    if (option=='gender'):
        fig.update_layout(title_text="Pourcentage de oui reçus selon le sexe", title_x=0.5)
    elif (option=='age'):
        fig.update_layout(title_text="Pourcentage de oui reçus selon l'âge", title_x=0.5)
    elif (option=='career_c'):
        fig.update_layout(title_text="Pourcentage de oui reçus selon le type de travail", title_x=0.5)

    return fig
```

Cette deuxième fonction a le même objectif : récupérer la valeur d'un radio bouton en entrée, et de renvoyer un graphique en sortie. Dans ce cas-là, il s'agit d'un histogramme.

Nous avons créé une troisième fonction callback qui a pour but d'afficher le contenu des pages en récupérant un url en entrée.

```
@app.callback(
    Output("page-content", "children"),
   [Input("url", "pathname")]
)
```
Si cet url correspond à "PresentationdesProfils", cela va afficher la page contenant les graphiques liés à la présentation des profils : 
```
    if pathname == "/PresentationdesProfils":
        return [
                html.H1('Présentation des Profils',
                        style={'textAlign':'center'}),
                html.Div(dcc.Graph(id="pie_age",
                        figure={'data': [go.Pie(values=dataage.value_counts(),labels=('[23-27]','[28-32]','[18-22]','[33-37]','38+'))],
                            'layout': {'title': 'Répartition des âges'}
                                }),
                    style={'border-top':'1px solid','border-bottom':'1px solid','border-left':'1px solid','float':'left','width':'30%','height':'500px'}),

                html.Div(dcc.Graph(id="hist_metier",
                        figure={'data': [go.Histogram(histfunc="count",x=datacar)],
                            'layout': {'title': 'Répartition des métiers'}
                                }),
                    style={'border' : '1px black solid','float':'left','width':'69%','height':'500px'}),]
```
Si l'url contient "ProfilType", cela va afficher les graphiques liés au profil type des participantes. Nous pouvons cocher par exemple un radio boutton, et la valeur sera récupéré par la fonction callback et renvoyée pour modifier le graphique selon cette valeur, comme expliqué précédemment :
```
    elif pathname == "/ProfilType":
        return [
                html.Div(children=[
                    html.Label('Choisissez un critère'),
                    dcc.RadioItems(id='select_input',
                        options=[
                                {'label': 'Sexe', 'value': 'gender'},
                                {'label': 'Travail', 'value': 'career_c'},
                                {'label': 'Age', 'value': 'age'}
                                ],className='select_categ',
                         value='gender'),

                    dcc.Graph(id='graph_output')
                ],style={'border-top':'1px solid','border-right':'1px solid','border-left':'1px solid','float':'left','width':'99%','height':'600px'}),
                
                html.Div(children=[
                    html.Label('Choisissez un critère'),
                    dcc.Dropdown(id='dropdown_input',
                                options=[{'label': 'Attractive', 'value': 'attr1_1'},
                                {'label': 'Sincere', 'value': 'sinc1_1'},
                                {'label': 'Intelligent', 'value': 'intel1_1'},
                                {'label': 'Fun', 'value': 'fun1_1'},
                                {'label': 'Ambitious', 'value': 'amb1_1'},
                                {'label': ' Has shared interests/hobbies', 'value': 'shar1_1'}],
                                 value='attr1_1',style={'width':'50%'}),

                    dcc.RadioItems(id='radio_input',
                                   options=[
                                       {'label': 'Sexe', 'value': 'gender'},
                                       {'label': 'Age', 'value': 'age'}
                                   ],
                                   value='gender'),
                    
                    dcc.Graph(id='boxplot_output')
                ],
                    style={'border' : '1px black solid','float':'left','width':'99%','height':'600px'}),html.Hr()]
```